In [1]:
import sys
sys.path.append("programme")
import pandas as pd 
from math import ceil,pi,floor
import os
df_cellule_general = pd.read_csv(os.path.join("..","fichier_sortie","fichier_7_21700.csv"))
import interface.interface

def index_str_to_int(df):
    index_prix_quantiter_int = []
    index_prix_quantiter_str =  df.index
    for index in index_prix_quantiter_str:
        try:
            index_prix_quantiter_int.append(int(index))
        except:
            None
    index_prix_quantiter_int.sort()
    return index_prix_quantiter_int
    
def detectors_quantiter_reduction(cellule):
    index_ligne = index_str_to_int(cellule)
    print("salut:",len(index_ligne))
    nombre_cellule = cellule["nb_cellule"]
    int_index_quantiter_eligible = 0
    str_index_quantiter_eligible = "prix_regulier"
    if len(index_ligne) == 1:
         if index_ligne < nombre_cellule:
                str_index_quantiter_eligible = str(index_ligne[0])
                return str_index_quantiter_eligible
    else:
        for i in index_ligne:
            if i < nombre_cellule:
                int_index_quantiter_eligible = i
            else:
                str_index_quantiter_eligible = str(int_index_quantiter_eligible)
                if str_index_quantiter_eligible == "0":
                    print(index_ligne)
                    str_index_quantiter_eligible = "prix_regulier"
    return str_index_quantiter_eligible

def prix_calculator():
    tour_de_boucle = 0
    df_cellule_general["prix_total"] = 0
    df_cellule_general["reduction"] = False
    for i in range(len(df_cellule_general)):
        print(tour_de_boucle)
        tour_de_boucle +=1
        cellule =  df_cellule_general.loc[i]
        print(isinstance(cellule, pd.Series))
        cellule.index
        cellule =  df_cellule_general.loc[i][df_cellule_general.loc[i].notna()]
        index_valide = detectors_quantiter_reduction(cellule)
        print("index valide:",index_valide)
        df_cellule_general["prix_total"].loc[i] = df_cellule_general[index_valide].loc[i]*df_cellule_general["nb_cellule"].loc[i]
        if index_valide != "prix_regulier":
            df_cellule_general["reduction"].loc[i] = True

            
        


In [3]:




VOLTAGE_NOMINAL = 3,6


#utilisateur_parametre pour calculer
utilisateur_voltage = 48
utilisateur_capaciter = 800
amperage_batterie = ceil(utilisateur_capaciter/utilisateur_voltage)

#option de trie : 
utilisateur_courant_de_decharge_max_ampere = 30
surface_diponible_cm_carres =  30000/100
separateur_cellule_surface_mm = 5
option_trie=[["puissance_max_A",utilisateur_courant_de_decharge_max_ampere],["cm²_par_cellule_avec_separateur",surface_diponible_cm_carres]]


df_cellule_general["nb_parallele"] = df_cellule_general["capaciter_ah"].apply(lambda capaciter_ah: ceil(amperage_batterie/capaciter_ah))
df_cellule_general["nb_serie"] = df_cellule_general["Tension nominale"].apply(lambda tension_nominal: round(utilisateur_voltage/tension_nominal))
df_cellule_general["tension_nominale_total"] = df_cellule_general["nb_serie"]*df_cellule_general["Tension nominale"]
df_cellule_general["capaciter_total"] =df_cellule_general["tension_nominale_total"]*amperage_batterie

df_cellule_general["nb_cellule"] = df_cellule_general["nb_parallele"]*df_cellule_general["nb_serie"]
#recuperer la colonne non nul
    
prix_calculator()
#calcule surface que la pile_seul a besoins
df_cellule_general["cm²_par_cellule"] =((df_cellule_general["Diamètre - mm"]/2)**2*pi)/100
df_cellule_general["cm²_total_sans_separateur"] = df_cellule_general["cm²_par_cellule"]*df_cellule_general["nb_cellule"]

#surface que la pile a besoins si séparateur
df_cellule_general["cm²_par_cellule_avec_separateur"] =(((df_cellule_general["Diamètre - mm"]+separateur_cellule_surface_mm*2)/2)**2*pi)/100
df_cellule_general["cm²_total_avec_separateur"] = df_cellule_general["cm²_par_cellule_avec_separateur"]*df_cellule_general["nb_cellule"]

#puissance_maximal de la batterie
df_cellule_general["puissance_max_A"] = df_cellule_general["Courant de décharge - A"]*df_cellule_general["nb_parallele"]
df_cellule_general["puissance_max_W"] = df_cellule_general["puissance_max_A"]*df_cellule_general["tension_nominale_total"]

df_cellule_general["poid_total_kg"] = (df_cellule_general["nb_cellule"]*df_cellule_general["Poids - g"])/1000
df_cellule_general

df_cellule_disponible = df_cellule_general[["nom","reduction","liens","Diamètre - mm","Hauteur - mm","nb_serie","nb_parallele","tension_nominale_total","capaciter_ah","capaciter_total","nb_cellule","prix_total","cm²_total_sans_separateur","cm²_total_avec_separateur","puissance_max_A","puissance_max_W","poid_total_kg"]][df_cellule_general["disponibiliter"]]

df_cellule_disponible.sort_values(["nb_cellule","prix_total"]).to_csv(os.path.join("..","fichier_sortie","cellule_trier.csv"))

df_cellule_disponible


0
True
salut: 4
index valide: 10
1
True
salut: 3
index valide: 30
2
True
salut: 3
index valide: 20
3
True
salut: 4
index valide: 30
4
True
salut: 1
index valide: 10
5
True
salut: 5
index valide: 50
6
True
salut: 3
index valide: 50
7
True
salut: 3
index valide: 50
8
True
salut: 3
index valide: 50
9
True
salut: 4
index valide: 30
10
True
salut: 4
index valide: 30
11
True
salut: 0
index valide: prix_regulier
12
True
salut: 3
index valide: 30
13
True
salut: 0
index valide: prix_regulier
14
True
salut: 0
index valide: prix_regulier
15
True
salut: 3
index valide: 50
16
True
salut: 0
index valide: prix_regulier
17
True
salut: 4
index valide: 60
18
True
salut: 5
index valide: 60
19
True
salut: 0
index valide: prix_regulier
20
True
salut: 1
index valide: 10
21
True
salut: 0
index valide: prix_regulier
22
True
salut: 4
index valide: 50
23
True
salut: 3
index valide: 10
24
True
salut: 4
index valide: 20
25
True
salut: 0
index valide: prix_regulier
26
True
salut: 3
index valide: 20
27
True
salut: 

C:\Users\vigre\AppData\Local\Temp\ipykernel_26008\2988567762.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cellule_general["prix_total"].loc[i] = df_cellule_general[index_valide].loc[i]*df_cellule_general["nb_cellule"].loc[i]
C:\Users\vigre\AppData\Local\Temp\ipykernel_26008\2988567762.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cellule_general["reduction"].loc[i] = True


,nom,reduction,liens,Diamètre - mm,Hauteur - mm,nb_serie,nb_parallele,tension_nominale_total,capaciter_ah,capaciter_total,nb_cellule,prix_total,cm²_total_sans_separateur,cm²_total_avec_separateur,puissance_max_A,puissance_max_W,poid_total_kg
1,Samsung INR21700-30T 3000mAh - 35A CLEAR WRAP ...,True,https://www.nkon.nl/fr/rechargeable/li-ion/217...,21.00,"70,10",13,6,46.8,3.00,795.6,78,159.90,270.161260,588.718755,210.0,9828.00,5.304
2,Sanyo NCR20700A 3300mAh - 30A NO WRAP,True,https://www.nkon.nl/fr/rechargeable/li-ion/217...,20.00,"70,00",13,6,46.8,3.30,795.6,78,354.90,245.044227,551.349511,180.0,8424.00,5.304
3,Samsung INR21700-48X 4650mAh - 9.6A NO WRAP,True,https://www.nkon.nl/fr/rechargeable/li-ion/217...,21.00,"71,00",13,4,46.8,4.65,795.6,52,184.60,180.107507,392.479170,38.4,1797.12,3.536
4,Nitecore 21700 NL2150HPi 5000mAh - 15A,True,https://www.nkon.nl/fr/rechargeable/li-ion/217...,21.90,"77,20",13,4,46.8,5.00,795.6,52,1141.40,195.876103,415.599093,60.0,2808.00,3.796
6,Samsung INR21700-33J 3270mAh - 6.4A - Single Tag,True,https://www.nkon.nl/fr/rechargeable/li-ion/217...,21.10,"70,15",13,6,46.8,3.27,795.6,78,132.60,272.740351,592.523067,38.4,1797.12,4.914
7,Samsung INR21700-33J 3270mAh - 6.4A - U-Tag,True,https://www.nkon.nl/fr/rechargeable/li-ion/217...,21.10,"70,15",13,6,46.8,3.27,795.6,78,144.30,272.740351,592.523067,38.4,1797.12,4.914
8,Samsung INR21700-33J 3270mAh - 6.4A - Z-Tag,True,https://www.nkon.nl/fr/rechargeable/li-ion/217...,21.10,"70,15",13,6,46.8,3.27,795.6,78,144.30,272.740351,592.523067,38.4,1797.12,4.914
10,Samsung INR21700-48X 4650mAh - 9.6A CLEAR WRAP,True,https://www.nkon.nl/fr/rechargeable/li-ion/217...,21.00,"71,00",13,4,46.8,4.65,795.6,52,195.00,180.107507,392.479170,38.4,1797.12,3.588
11,Olight 21700 5000mAh batterie pour Seeker 2 Pro,False,https://www.nkon.nl/fr/rechargeable/li-ion/217...,22.00,"74,00",13,4,46.8,5.00,795.6,52,1557.40,197.669010,418.208814,60.0,2808.00,4.264
13,XTAR 21700 4900mAh (protégé) - 10A,False,https://www.nkon.nl/fr/rechargeable/li-ion/217...,21.00,"74,60",13,4,46.8,4.90,795.6,52,621.40,180.107507,392.479170,40.0,1872.00,3.744


In [235]:


df_cellule_general.columns



Index(['nom', 'liens', 'disponibiliter', 'prix_regulier', 'remis_aditionnelle',
       '10', '100', '200', '400', 'EAN / GTIN', 'Poids - g', 'Marque', 'Model',
       'Taille de la batterie', 'Batterie chimique', 'Batterie',
       'Tension nominale', 'capaciter_ah', 'Version batterie',
       'Courant de décharge - A', 'Protection des circuits', 'Hauteur - mm',
       'Diamètre - mm', '30', '130', '260', '390', '20', '50', '60', '128',
       '256', '512', 'Inclus', '25', '125', '250', '600', '520',
       'nb_parallele', 'nb_serie', 'tension_nominale_total', 'capaciter_total',
       'nb_cellule', 'prix_total', 'reduction', 'cm²_par_cellule',
       'cm²_total_sans_separateur', 'cm²_par_cellule_avec_separateur',
       'cm²_total_avec_separateur', 'puissance_max_A', 'puissance_max_W',
       'poid_total_kg'],
      dtype='object')